## Feature Extraction
Extract features from raw data.

Here, we first start with extracting new features from already existing variables.


In [38]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.width', 500)


def load():
    data = pd.read_csv("datasets/titanic.csv")
    return data


df = load()

In [39]:
##Binary features
df["NEW_CABIN_FLAG"] = df["Cabin"].isnull().astype('int') ##if NAN 1 else 0

df.groupby("NEW_CABIN_FLAG").agg({"Survived": "mean"})
##interestingly, if cabin info is missing, survival rate is lower

,Survived
NEW_CABIN_FLAG,
0,0.667
1,0.300


In [40]:
df.loc[((df["SibSp"] + df["Parch"]) > 0), "IS_ALONE"] = "NO"
df.loc[((df["SibSp"] + df["Parch"]) == 0), "IS_ALONE"] = "YES"

df.groupby("IS_ALONE").agg({"Survived": "mean"})

,Survived
IS_ALONE,
NO,0.506
YES,0.304


In [41]:
##Text based features

##character count
df["NEW_NAME_COUNT"] = df["Name"].str.len()

##word count
df["NEW_NAME_WORD_COUNT"] = df["Name"].apply(lambda x: len(str(x).split(" ")))

##any specific word in the name
df["NEW_NAME_DR"] = df["Name"].apply(lambda x: len([x for x in x.split() if x.startswith("Dr")]))

df.groupby("NEW_NAME_DR").agg({"Survived": ["mean", "count"]})

Survived      
                mean count
NEW_NAME_DR               
0              0.383   881
1              0.500    10

In [42]:
##Regex 

##extract title from name
df["NEW_NAME_TITLE"] = df["Name"].str.extract(' ([A-Za-z]+)\.', expand=False)

df["NEW_NAME_TITLE"].value_counts()


NEW_NAME_TITLE
Mr          517
Miss        182
Mrs         125
Master       40
Dr            7
Rev           6
Mlle          2
Major         2
Col           2
Countess      1
Capt          1
Ms            1
Sir           1
Lady          1
Mme           1
Don           1
Jonkheer      1
Name: count, dtype: int64

In [43]:
df[["NEW_NAME_TITLE", "Survived", "Age"]].groupby(["NEW_NAME_TITLE"]).agg({"Survived": ["mean", "count"], "Age": ["mean","count"]}) 

Survived          Age      
                   mean count   mean count
NEW_NAME_TITLE                            
Capt              0.000     1 70.000     1
Col               0.500     2 58.000     2
Countess          1.000     1 33.000     1
Don               0.000     1 40.000     1
Dr                0.429     7 42.000     6
Jonkheer          0.000     1 38.000     1
Lady              1.000     1 48.000     1
Major             0.500     2 48.500     2
Master            0.575    40  4.574    36
Miss              0.698   182 21.774   146
Mlle              1.000     2 24.000     2
Mme               1.000     1 24.000     1
Mr                0.157   517 32.368   398
Mrs               0.792   125 35.898   108
Ms                1.000     1 28.000     1
Rev               0.000     6 43.167     6
Sir               1.000     1 49.000     1